# 用 python 画动画
> "总结一下 python 画动画的方法"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Shan Jin
- categories: [python]

# 在 jupyter notebook 中展现动画
首先为了在 jupyter notebook 中展现动画，要先载入交互式绘图：

In [ ]:
%matplotlib notebook
#calling it a second time may prevent some graphics errors
%matplotlib notebook 

# 用 FuncAnimation 画动画
## 简单示例
利用 [FuncAnimation](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html) 可以实现 python 动画，下面是一个例子：

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()

# line, = ax.plot([])


x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x)

line, = ax.plot(x, y)
# plt.plot(x, y)

ax.set_xlim(0, 2*np.pi)
ax.set_ylim(-1.1, 1.1)


def animate(frame_num):
    y = np.sin(x + 2*np.pi * frame_num/100)
    line.set_data((x, y))
    return line

ani = FuncAnimation(fig, animate, frames=100, interval=5)
# plt.show()
# ani.save('ani.mp4',fps=100)
# ani.save('ani.gif', writer='imagemagick', fps=100)

其中 `frames=100` 是总帧数， `interval=5` 是每帧之间的间隔，单位是毫秒。
## 关于储存
`FuncAnimation` 返回对象可以直接存为 `.mp4` 格式的文件，例如上述示例中用 `ani.save('ani.mp4',fps=100)` 进行储存。由于 `matplotlib` 本身不支持 `.gif` 格式的储存，若要保存为 `.gif` 格式文件，必须安装外部软件来实现，比如 `imagemagick`. 在正确安装 [ImageMagick](https://www.imagemagick.org/script/index.php) 之后，可用它来实现 `.gif` 格式的存储，例如上述示例中，储存语句为 `ani.save('ani.gif', writer='imagemagick', fps=100)`.

> Note: 用 `plot` 函数作图时，用 `set_data` 更新数据；对于 `imshow` 或 `pcolormesh` 作图则用 `set_array` 来更新数据。

## pcolormesh 动画
下面是一个简单的 `pcolormesh` 动画 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

y = np.linspace(-np.pi, np.pi, 100)
x = np.linspace(-np.pi, np.pi, 100)
C = np.ones((100, 100)) * float('nan')

# intantiate empty plot (values = nan)
fig, ax = plt.subplots()
pcmesh = ax.pcolormesh(x, y, C, vmin=-1, vmax=1, shading='gouraud')

# generate some new data
X, Y = np.meshgrid(x, y)
def animate(frame_num):
    C = np.sin(X+2*np.pi * frame_num/100) 
    # ravel() converts C to a 1d-array
    pcmesh.set_array(C.ravel())
    return pcmesh

ani = FuncAnimation(fig, animate, frames=100, interval=3)

注意：`shading='gouraud'` 必须要加上，否则会报错:
```
ValueError: total size of new array must be unchanged
```
## 画动画的一般方法
从上面两个例子中可以看到，不同的画图函数更新数据的方法不同，当调用一些陌生的绘图指令时更是不知道如何更新数据。有种通用的方法是每次都清除面板、重新绘图，有点类似 `matlab` 里画动画的操作，示例如下：

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()

x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x)

line, = ax.plot(x, y)
# plt.plot(x, y)

ax.set_xlim(0, 2*np.pi)
ax.set_ylim(-1.1, 1.1)

def animate(frame_num, x, plot):
    y = np.sin(x + 2*np.pi * frame_num/100)
    ax.clear()
    plot = ax.plot(x, y)
    ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1.1, 1.1)
    return plot

plot_ani = FuncAnimation(fig, animate, frames=100 ,fargs=(x, line),interval=5)

# 用 imageio 创建 gif
当然咯，如果你觉得 `FuncAnimation` 的使用规则太过于复杂，那么还可以用一种极其简单的方法来画动画，就是把构成动画的每一帧图形都保存下来，然后利用 [imageio](https://imageio.readthedocs.io/en/stable/) 来创建 `.gif` 格式文件。

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

y = np.linspace(-np.pi, np.pi, 100)
x = np.linspace(-np.pi, np.pi, 100)
X, Y = np.meshgrid(x, y)

# save every frame of animation
for i in range(100):
    C = np.sin(X+2*np.pi * i/100) 
    fig, ax = plt.subplots()
    ax.pcolormesh(x, y, C, vmin=-1, vmax=1, shading='gouraud')
    plt.savefig('figs/'+str(i).zfill(3)+'.png')

# to create gif 
images = []
for i in range(100):
    filename = 'figs/'+str(i).zfill(3)+'.png'
    images.append(imageio.imread(filename))
imageio.mimsave('ani.gif', images, duration=0.01)

注意：`duration` 的单位是秒。